# Uncertainty Based Cost-Effective Labeling Methodology

In [16]:
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn as nn
import time
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import os
import natsort
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd
import numpy as np

# Device Setting

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class gpu_setting:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model

In [4]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        # identity mapping, input과 output의 feature map size, filter 수가 동일한 경우 사용.
        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        # projection mapping using 1x1conv
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=9, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [5]:
def ResNetParameters(model, train_dl, valid_dl):
    device = gpu_setting.device
    model = model.lower()
    if model == 'resnet34':
        model = resnet34().to(device)
    if model == 'resnet50':
        model = resnet50().to(device)
    if model == 'resnet101':
        model = resnet101().to(device)
        
    loss_func = nn.CrossEntropyLoss(reduction='sum')
    opt = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)
    
    # definc the training parameters
    params_train = {
        'num_epochs':5,
        'optimizer':opt,
        'loss_func':loss_func,
        'train_dl':train_dl, 
        'val_dl':valid_dl,
        'sanity_check':False,
        'lr_scheduler':lr_scheduler,
        'path2weights':'./trained_model/weights_original_res.pt'
    }
    return model, params_train

# Function

In [6]:
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects

def loss_batch(loss_func, output, target, opt=None):
    loss = loss_func(output, target)
    metric_b = metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        output = model(xb)

        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)

        running_loss += loss_b
        
        if metric_b is not None:
            running_metric += metric_b
        
        if sanity_check is True:
            break

    loss = running_loss / len_data
    metric = running_metric / len_data

    return loss, metric

def train_val(model, params, epoch):
    num_epochs=epoch
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    val_dl=params["val_dl"]
    sanity_check=params["sanity_check"]
    lr_scheduler=params["lr_scheduler"]
    path2weights=params["path2weights"]

    loss_history = {'train': [], 'val': []}
    metric_history = {'train': [], 'val': []}

    # # GPU out of memoty error
    # best_model_wts = copy.deepcopy(model.state_dict())

    best_loss = float('inf')

    start_time = time.time()

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print('Epoch {}/{}, current lr={}'.format(epoch, num_epochs-1, current_lr))

        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl, sanity_check)
        loss_history['val'].append(val_loss)
        metric_history['val'].append(val_metric)

        if val_loss < best_loss:
            best_loss = val_loss
            # best_model_wts = copy.deepcopy(model.state_dict())

            torch.save(model.state_dict(), path2weights)
            print('Copied best model weights!')
            print('Get best val_loss')

        lr_scheduler.step(val_loss)

        print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
        print('-'*10)

    # model.load_state_dict(best_model_wts)

    return model, loss_history, metric_history

In [7]:
class CustomDataSet():
    def __init__(self, main_dir, transform, num):
        self.main_dir = main_dir
        self.transform = transform
        self.num = num
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

In [17]:
class CustomSubset(Dataset):
    def __init__(self,Subset,transform=None):
        super(CustomSubset,self).__init__()
        self.Subset=Subset
        self.indices=Subset.indices
        self.transform=transform

    def __len__(self):
        return len(self.Subset)

    def __getitem__(self,idx):
        img,label=self.Subset[idx]
        if self.transform is not None:
            img=self.transform(img)
        return img,label

# csv

In [69]:
df_155655 = pd.read_csv('../csv/data_155655.csv')
label = pd.read_csv('../csv/data_172950.csv', index_col=0)

# Phase 1

## Training

In [70]:
# Initial train data(17,295)
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [71]:
print(len(train_data), len(valid_data))

13836 3459


In [72]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [73]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [74]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.364726, val loss: 0.355461, accuracy: 91.93, time: 1.5254 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.229135, val loss: 0.206227, accuracy: 93.52, time: 2.3221 min
----------
Epoch 2/4, current lr=0.001
train loss: 0.203866, val loss: 0.255207, accuracy: 92.28, time: 3.1225 min
----------
Epoch 3/4, current lr=0.001
train loss: 0.182893, val loss: 0.309487, accuracy: 91.12, time: 3.9249 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.167609, val loss: 0.222912, accuracy: 93.78, time: 4.7345 min
----------


## Predict

### Set 1 Data Load

In [75]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

### Set 1 data predict & Softmax

In [76]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [77]:
len(pred_list)

17295

In [78]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [79]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [80]:
engineer_label_count = []
wrong_cnt_list = []
real_label_list = list(df_155655.loc[:17294]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 3152563.57it/s]


46

In [81]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [82]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130]
[46]


## Append Set 1 Data to Train Data

In [84]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 2

## Training

In [85]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [86]:
print(len(train_data), len(valid_data))

27672 6918


In [87]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [90]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [91]:
model, loss_hist, metric_hist = train_val(model, params_train, 7)

Epoch 0/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.344499, val loss: 0.325057, accuracy: 91.44, time: 1.6534 min
----------
Epoch 1/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.209329, val loss: 0.239570, accuracy: 93.21, time: 3.2910 min
----------
Epoch 2/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.172744, val loss: 0.149862, accuracy: 94.84, time: 4.9431 min
----------
Epoch 3/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.149204, val loss: 0.138496, accuracy: 95.79, time: 6.5751 min
----------
Epoch 4/6, current lr=0.001
train loss: 0.134793, val loss: 0.161303, accuracy: 95.33, time: 8.2285 min
----------
Epoch 5/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.119900, val loss: 0.134303, accuracy: 95.94, time: 9.8890 min
----------
Epoch 6/6, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 

## Predict

### Set 2 Data Load

In [94]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

In [95]:
len(test_loader.dataset)

17295

### Set 2 Data Predict & Softmax

In [100]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [101]:
len(pred_list)

17295

In [102]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [108]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [116]:
real_label_list = list(df_155655.loc[17295*1:17295*2-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2892133.31it/s]


41

In [117]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [118]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998]
[46, 41]


## Append Set 2 Data to Train data

In [119]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 3

## Training

In [127]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [128]:
print(len(train_data), len(valid_data))

41508 10377


In [129]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [130]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [131]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.341639, val loss: 0.236236, accuracy: 92.72, time: 2.4815 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.200848, val loss: 0.202983, accuracy: 93.87, time: 4.9023 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.168646, val loss: 0.163759, accuracy: 94.99, time: 7.3422 min
----------
Epoch 3/4, current lr=0.001
train loss: 0.140114, val loss: 0.168486, accuracy: 94.57, time: 9.7998 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.125964, val loss: 0.225973, accuracy: 93.72, time: 12.2306 min
----------


## Predict

### Set 3 Data Load

In [132]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

In [133]:
len(test_loader.dataset)

17295

### Set 3 Data Predict & Softmax

In [134]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [135]:
len(pred_list)

17295

In [136]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [137]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [138]:
real_label_list = list(df_155655.loc[17295*2:17295*3-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2478999.65it/s]


19

In [139]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [140]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353]
[46, 41, 19]


## Append Set 3 Data to Train Data

In [146]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 4

## Training

In [147]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [148]:
print(len(train_data), len(valid_data))

55344 13836


In [149]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [150]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [151]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.289353, val loss: 0.212728, accuracy: 93.40, time: 3.3250 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.185630, val loss: 0.134281, accuracy: 95.23, time: 6.6139 min
----------
Epoch 2/4, current lr=0.001
train loss: 0.155860, val loss: 0.144121, accuracy: 95.46, time: 9.9798 min
----------
Epoch 3/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.135188, val loss: 0.125400, accuracy: 96.18, time: 13.2296 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.120916, val loss: 0.140377, accuracy: 95.58, time: 16.5038 min
----------


## Predict

### Set 4 Data Load

In [152]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

In [153]:
len(test_loader.dataset)

17295

### Set 4  Data Predict & Softmax

In [154]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [155]:
len(pred_list)

17295

In [156]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [157]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [158]:
real_label_list = list(df_155655.loc[17295*3:17295*4-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2892248.62it/s]


36

In [159]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [160]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112]
[46, 41, 19, 36]


## Append Set 4 Data to Train Data

In [161]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 5

## Training

In [162]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [163]:
print(len(train_data), len(valid_data))

69180 17295


In [164]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [165]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [166]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.256166, val loss: 0.148044, accuracy: 95.27, time: 4.1261 min
----------
Epoch 1/4, current lr=0.001
train loss: 0.156794, val loss: 0.161723, accuracy: 95.17, time: 10.8184 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.136792, val loss: 0.113433, accuracy: 96.31, time: 15.3467 min
----------
Epoch 3/4, current lr=0.001
train loss: 0.115320, val loss: 0.134220, accuracy: 96.11, time: 19.4349 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.099573, val loss: 0.160896, accuracy: 95.72, time: 23.4916 min
----------


## Predict

### Set 5 Data Load

In [167]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

In [168]:
len(test_loader.dataset)

17295

### Set 5 Data Predict & Softmax

In [169]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [170]:
len(pred_list)

17295

In [171]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [172]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [173]:
real_label_list = list(df_155655.loc[17295*4:17295*5-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2891326.39it/s]


45

In [174]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [175]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112, 2522]
[46, 41, 19, 36, 45]


## Append Set 5 to Train Data

In [176]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 6

## Training

In [177]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [179]:
print(len(train_data), len(valid_data))

83016 20754


In [180]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [183]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [184]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.246027, val loss: 0.177681, accuracy: 94.55, time: 4.8952 min
----------
Epoch 1/4, current lr=0.001
train loss: 0.153302, val loss: 0.464455, accuracy: 86.83, time: 10.1070 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.128450, val loss: 0.117121, accuracy: 96.42, time: 15.0669 min
----------
Epoch 3/4, current lr=0.001
train loss: 0.111389, val loss: 0.125228, accuracy: 96.46, time: 19.7835 min
----------
Epoch 4/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.098176, val loss: 0.097652, accuracy: 97.11, time: 24.7524 min
----------


## Predict

### Set 6 Data Load

In [185]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

In [186]:
len(test_loader.dataset)

17295

### Set 6 Data Predict & Softmax

In [187]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [188]:
len(pred_list)

17295

In [189]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [190]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [191]:
real_label_list = list(df_155655.loc[17295*5:17295*6-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2892133.31it/s]


74

In [192]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [193]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112, 2522, 1834]
[46, 41, 19, 36, 45, 74]


## Appned Set 6 Data to Train data

In [194]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 7

## Training

In [195]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [196]:
print(len(train_data), len(valid_data))

96852 24213


In [197]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [198]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [199]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.220845, val loss: 0.153382, accuracy: 95.22, time: 6.6367 min
----------
Epoch 1/4, current lr=0.001
train loss: 0.138895, val loss: 0.194730, accuracy: 93.84, time: 13.1283 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.113683, val loss: 0.105405, accuracy: 96.60, time: 19.5822 min
----------
Epoch 3/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.099146, val loss: 0.080264, accuracy: 97.42, time: 25.2131 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.084296, val loss: 0.088859, accuracy: 97.09, time: 30.8535 min
----------


## Predict

### Set 7 Data Load

In [200]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

### Set 7 Data Predict & Softmax

In [201]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [202]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [203]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)
for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

In [204]:
real_label_list = list(df_155655.loc[17295*6:17295*7-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2478999.65it/s]


70

In [205]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [206]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112, 2522, 1834, 2644]
[46, 41, 19, 36, 45, 74, 70]


## Append Set 7 Data to Train Data

In [207]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 8

## Training

In [208]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [209]:
print(len(train_data), len(valid_data))

110688 27672


In [210]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [211]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [212]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.219998, val loss: 0.165572, accuracy: 95.23, time: 7.1483 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.140101, val loss: 0.114836, accuracy: 96.12, time: 14.1868 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.116261, val loss: 0.110472, accuracy: 96.52, time: 20.6697 min
----------
Epoch 3/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.096622, val loss: 0.087698, accuracy: 97.12, time: 27.3044 min
----------
Epoch 4/4, current lr=0.001
train loss: 0.084356, val loss: 0.104829, accuracy: 96.70, time: 33.9670 min
----------


## Predict

### Set 8 Data Load

In [213]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

### Set 8 Data Predict & Softmax

In [214]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [215]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [216]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)

for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [217]:
real_label_list = list(df_155655.loc[17295*7:17295*8-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2665263.90it/s]


18

In [218]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [219]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112, 2522, 1834, 2644, 1792]
[46, 41, 19, 36, 45, 74, 70, 18]


## Add Set 8 Data to Train Data

In [220]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Phase 9

## Training

In [221]:
train_dir = '../Data/Data_17295/Labeled/'
train_folder_dataset = dset.ImageFolder(root=train_dir)

# train & valid split
train_data_len = int(len(train_folder_dataset)*0.8)
valid_data_len = len(train_folder_dataset) - train_data_len
train_data, valid_data = random_split(train_folder_dataset, [train_data_len, valid_data_len])

In [222]:
print(len(train_data), len(valid_data))

124524 31131


In [223]:
train_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])

train_data = CustomSubset(train_data, train_transformation)
valid_data = CustomSubset(valid_data, train_transformation)

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

In [224]:
model, params_train = ResNetParameters('resnet50', train_dl, valid_dl)

In [225]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.201428, val loss: 0.161952, accuracy: 95.04, time: 7.1705 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.126477, val loss: 0.124955, accuracy: 96.04, time: 14.5604 min
----------
Epoch 2/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.104695, val loss: 0.102634, accuracy: 96.57, time: 22.0511 min
----------
Epoch 3/4, current lr=0.001
train loss: 0.089598, val loss: 0.137079, accuracy: 95.85, time: 29.5040 min
----------
Epoch 4/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.077900, val loss: 0.099528, accuracy: 96.82, time: 37.0014 min
----------


## Predict

### Set 9 Data Load

In [227]:
num = 0
test_dir = '../Data/Data_17295/Unlabeled/'
test_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                ])
my_dataset = CustomDataSet(test_dir, transform=test_transformation, num=0)
test_loader = DataLoader(my_dataset, shuffle=False)

all_imgs = os.listdir(test_dir)
all_imgs = natsort.natsorted(all_imgs)[17295*num:17295*(num+1)]

### Set 9 Data Predict & Softmax

In [228]:
pred_list =[]
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [229]:
max_list = []
for i in range(0,17295):
    max_list.append(float(pred_list[i][0].max()))

### Uncertainty

In [230]:
cnt = 0
label_list = []
img_folder = natsort.natsorted(all_imgs)

for i in range(0,17295):
    if pred_list[i][0].max()<0.99:
        idx = int(img_folder[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

### Mislabelings & Engineer Cost

In [231]:
real_label_list = list(df_155655.loc[17295*8:17295*9-1]['failureNum'].values)
wrong_cnt = 0
for i in tqdm(range(0,17295)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|███████████████████████████████████████████████████████████████████████| 17295/17295 [00:00<00:00, 2892363.94it/s]


90

In [232]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

## Result

In [233]:
print(engineer_label_count)
print(wrong_cnt_list)

[6130, 4998, 5353, 2112, 2522, 1834, 2644, 1792, 1947]
[46, 41, 19, 36, 45, 74, 70, 18, 90]


## Add Set 9 Data to Train Data

In [234]:
for i in range(0,17295):
    shutil.move('../Data/Data_17295/Unlabeled/{0}'.format(img_folder[i]),'../Data/Data_17295/Labeled/{0}/{1}'.format(label_list[i],img_folder[i]))

# Final Result
- Mislabelings : 439 / 155,655(0.28%)
    - per phase : 46 / 41 / 19 / 36 / 45 / 74 / 70 / 18 / 90
- Engineer Cost : 29,332 / 155,655(18.84%)
    - per phase : 6130 / 4998 / 5353 / 2112 / 2522 / 1834 / 2644 / 1792 / 1947